##Install necessary packages

In [29]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
cleaned = pd.read_excel("/content/drive/My Drive/UIUC Material/BADM 550 (AWG)/data_cleaned.xlsx")
sec = pd.read_excel("/content/drive/My Drive/UIUC Material/BADM 550 (AWG)/merge4_table.xlsx")

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [31]:
# prompt:  group by Participation.#. after that, count participation based on cleaned table. and add three columns, named participation in year 20##, if Participation.# appear in that year,  give value 1, if not 0

# Group by 'Participation.#' and count occurrences
participation_counts = cleaned.groupby('Participation.#').size().reset_index(name='participation_count')

# Create new columns for participation in specific years
years = [2017, 2018, 2019, 2020, 2021, 2022, 2023]  # Add or modify the years as needed

for year in years:
    participation_counts[f'participation_in_{year}'] = 0

# Iterate through the grouped data and update the year columns
for index, row in participation_counts.iterrows():
    participation_id = row['Participation.#']
    for year in years:
        if cleaned[cleaned['Participation.#'] == participation_id]['year_submitted'].isin([year]).any():
            participation_counts.loc[index, f'participation_in_{year}'] = 1

# Print or further process the updated DataFrame
participation_counts

,Participation.#,participation_count,participation_in_2017,participation_in_2018,participation_in_2019,participation_in_2020,participation_in_2021,participation_in_2022,participation_in_2023
0,1004265,17,0,1,1,1,1,1,1
1,1004287,3,0,1,1,0,0,0,0
2,1005804,4,0,1,1,0,0,1,1
3,1008463,2,0,0,1,0,0,0,1
4,1008537,3,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
4179,5474511,3,0,1,1,0,1,0,0
4180,5545543,8,0,1,1,1,1,1,1
4181,7563738,1,0,1,0,0,0,0,0
4182,7567240,7,0,1,1,1,1,0,1


In [32]:
# prompt: I want to join both participation_counts table and sec table by Participation.#

# Merge the two dataframes
pre_learning_data = pd.merge(participation_counts, sec, on='Participation.#', how='left')

# Display the merged dataframe
pre_learning_data

,Participation.#,participation_count,participation_in_2017,participation_in_2018,participation_in_2019,participation_in_2020,participation_in_2021,participation_in_2022,participation_in_2023,sum_total,duration,St,average_period,class
0,1004265,17,0,1,1,1,1,1,1,780,0,OK,37.705882,Religious Organization
1,1004287,3,0,1,1,0,0,0,0,330,0,OK,11.000000,Association/Foundation
2,1005804,4,0,1,1,0,0,1,1,240,0,MO,3.000000,Association/Foundation
3,1008463,2,0,0,1,0,0,0,1,120,0,MO,8.500000,Religious Organization
4,1008537,3,0,1,1,0,1,0,0,240,0,MO,2.000000,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4179,5474511,3,0,1,1,0,1,0,0,210,0,AR,2.333333,Religious Organization
4180,5545543,8,0,1,1,1,1,1,1,450,0,MO,23.875000,Other
4181,7563738,1,0,1,0,0,0,0,0,60,0,MO,1.000000,Education Institution
4182,7567240,7,0,1,1,1,1,0,1,360,0,OK,4.142857,Religious Organization


In [33]:
  # prompt: St is category type data, how to normalize the St in pre_learning_data

# Assuming pre_learning_data is the DataFrame from the previous code
pre_learning_data = pd.get_dummies(pre_learning_data, columns=['St'], prefix='St')
pre_learning_data = pd.get_dummies(pre_learning_data, columns=['class'], prefix='class')
pre_learning_data

,Participation.#,participation_count,participation_in_2017,participation_in_2018,participation_in_2019,participation_in_2020,participation_in_2021,participation_in_2022,participation_in_2023,sum_total,...,St_WV,St_WY,class_Association/Foundation,class_Community Club,class_Education Institution,class_Food Bank,class_Healtcare and Public Safety,class_Library,class_Other,class_Religious Organization
0,1004265,17,0,1,1,1,1,1,1,780,...,False,False,False,False,False,False,False,False,False,True
1,1004287,3,0,1,1,0,0,0,0,330,...,False,False,True,False,False,False,False,False,False,False
2,1005804,4,0,1,1,0,0,1,1,240,...,False,False,True,False,False,False,False,False,False,False
3,1008463,2,0,0,1,0,0,0,1,120,...,False,False,False,False,False,False,False,False,False,True
4,1008537,3,0,1,1,0,1,0,0,240,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4179,5474511,3,0,1,1,0,1,0,0,210,...,False,False,False,False,False,False,False,False,False,True
4180,5545543,8,0,1,1,1,1,1,1,450,...,False,False,False,False,False,False,False,False,True,False
4181,7563738,1,0,1,0,0,0,0,0,60,...,False,False,False,False,True,False,False,False,False,False
4182,7567240,7,0,1,1,1,1,0,1,360,...,False,False,False,False,False,False,False,False,False,True


In [34]:
# prompt: names of columns in pre_learning_data

# Assuming pre_learning_data is the DataFrame from the previous code
pre_learning_data.columns

Index(['Participation.#', 'participation_count', 'participation_in_2017',
       'participation_in_2018', 'participation_in_2019',
       'participation_in_2020', 'participation_in_2021',
       'participation_in_2022', 'participation_in_2023', 'sum_total',
       'duration', 'average_period', 'St_AL', 'St_AR', 'St_CT', 'St_FL',
       'St_GA', 'St_IA', 'St_IL', 'St_IN', 'St_KS', 'St_KY', 'St_LA', 'St_MI',
       'St_MN', 'St_MO', 'St_MS', 'St_NE', 'St_NM', 'St_OH', 'St_OK', 'St_PA',
       'St_SD', 'St_TN', 'St_TX', 'St_WI', 'St_WV', 'St_WY',
       'class_Association/Foundation', 'class_Community Club',
       'class_Education Institution', 'class_Food Bank',
       'class_Healtcare and Public Safety', 'class_Library', 'class_Other',
       'class_Religious Organization'],
      dtype='object')

##Random Forest Model

In [35]:
RFDataX = pre_learning_data[['participation_count', 'participation_in_2017',
       'participation_in_2018', 'participation_in_2019',
       'participation_in_2020', 'participation_in_2021',
       'participation_in_2022', 'sum_total',
       'duration', 'average_period', 'St_AL', 'St_AR', 'St_CT', 'St_FL',
       'St_GA', 'St_IA', 'St_IL', 'St_IN', 'St_KS', 'St_KY', 'St_LA', 'St_MI',
       'St_MN', 'St_MO', 'St_MS', 'St_NE', 'St_NM', 'St_OH', 'St_OK', 'St_PA',
       'St_SD', 'St_TN', 'St_TX', 'St_WI', 'St_WV', 'St_WY',
       'class_Association/Foundation', 'class_Community Club',
       'class_Education Institution', 'class_Food Bank',
       'class_Healtcare and Public Safety', 'class_Library', 'class_Other',
       'class_Religious Organization']]
RFDataY = pre_learning_data[['participation_in_2023']]

In [36]:
RFX_train, RFX_test, RFY_train, RFY_test = train_test_split(RFDataX, RFDataY, test_size=0.3, random_state=1)

In [37]:
from sklearn.ensemble import RandomForestClassifier
rfclf = RandomForestClassifier(n_estimators = 8, criterion = 'gini', random_state = 1, max_depth= 50)
# n_estimators refers to the number of decision trees to be constructed
rfclf.fit(RFX_train, np.ravel(RFY_train))

RandomForestClassifier(max_depth=50, n_estimators=8, random_state=1)

In [38]:
RFY_pred = rfclf.predict(RFX_test)
print('Accuracy of Random Forest classifier on test set: {:.2f}'.format(rfclf.score(RFX_test, RFY_test)))

Accuracy of Random Forest classifier on test set: 0.92


In [39]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(RFY_test, RFY_pred)
print(confusion_matrix)

[[740  23]
 [ 72 421]]


## Navie Bayes

In [40]:
NBDataX = pre_learning_data[['participation_count', 'participation_in_2017',
       'participation_in_2018', 'participation_in_2019',
       'participation_in_2020', 'participation_in_2021',
       'participation_in_2022', 'sum_total',
       'duration', 'average_period', 'St_AL', 'St_AR', 'St_CT', 'St_FL',
       'St_GA', 'St_IA', 'St_IL', 'St_IN', 'St_KS', 'St_KY', 'St_LA', 'St_MI',
       'St_MN', 'St_MO', 'St_MS', 'St_NE', 'St_NM', 'St_OH', 'St_OK', 'St_PA',
       'St_SD', 'St_TN', 'St_TX', 'St_WI', 'St_WV', 'St_WY',
       'class_Association/Foundation', 'class_Community Club',
       'class_Education Institution', 'class_Food Bank',
       'class_Healtcare and Public Safety', 'class_Library', 'class_Other',
       'class_Religious Organization']]
NBDataY = pre_learning_data[['participation_in_2023']]

In [41]:
NBX_train, NBX_test, NBY_train, NBY_test = train_test_split(NBDataX, NBDataY, test_size=0.3, random_state=1)

In [42]:
#Create a Classifier and Train the model
#BERNmodel = BernoulliNB()
GAUSSmodel = GaussianNB()

# If the other variables were continuous, I would use GaussianNB()
#BERNmodel.fit(NBX_train, np.ravel(NBY_train))
GAUSSmodel.fit(NBX_train, np.ravel(NBY_train))

GaussianNB()

In [43]:
from sklearn.metrics import confusion_matrix

#Predict Bernoulli Output
#NBERNY_pred = BERNmodel.predict(NBX_test)
NGaussY_pred = GAUSSmodel.predict(NBX_test)

#NBERNConfusion_Matrix = confusion_matrix(NBY_test, NBERNY_pred)
NGaussConfusion_Matrix = confusion_matrix(NBY_test, NGaussY_pred)

#print(NBERNConfusion_Matrix)
print(NGaussConfusion_Matrix)

[[554 209]
 [ 47 446]]


In [44]:
#BERNmodel.score(NBX_test, NBY_test)
GAUSSmodel.score(NBX_test, NBY_test)

0.7961783439490446

##Logistic Regression

In [45]:
LRDataX = pre_learning_data[['participation_count', 'participation_in_2017',
       'participation_in_2018', 'participation_in_2019',
       'participation_in_2020', 'participation_in_2021',
       'participation_in_2022', 'sum_total',
       'duration', 'average_period', 'St_AL', 'St_AR', 'St_CT', 'St_FL',
       'St_GA', 'St_IA', 'St_IL', 'St_IN', 'St_KS', 'St_KY', 'St_LA', 'St_MI',
       'St_MN', 'St_MO', 'St_MS', 'St_NE', 'St_NM', 'St_OH', 'St_OK', 'St_PA',
       'St_SD', 'St_TN', 'St_TX', 'St_WI', 'St_WV', 'St_WY',
       'class_Association/Foundation', 'class_Community Club',
       'class_Education Institution', 'class_Food Bank',
       'class_Healtcare and Public Safety', 'class_Library', 'class_Other',
       'class_Religious Organization']]
LRDataY = pre_learning_data[['participation_in_2023']]

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

LRDataX_train, LRDataX_test, LRDataY_train, LRDataY_test = train_test_split(LRDataX, LRDataY, test_size=0.3, random_state=1)
awg_logisticreg = LogisticRegression(solver='lbfgs')
awg_logisticreg.fit(LRDataX_train, np.ravel(LRDataY_train))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [47]:
LRY_pred = awg_logisticreg.predict(LRDataX_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(awg_logisticreg.score(LRDataX_test, LRDataY_test)))

Accuracy of logistic regression classifier on test set: 0.92


## SVM

In [48]:
import matplotlib.pyplot as plt
plt.rc("font", size=13)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.inspection import DecisionBoundaryDisplay
%matplotlib inline

In [49]:
SVMDataX = pre_learning_data[['participation_count', 'participation_in_2017',
       'participation_in_2018', 'participation_in_2019',
       'participation_in_2020', 'participation_in_2021',
       'participation_in_2022', 'sum_total',
       'duration', 'average_period', 'St_AL', 'St_AR', 'St_CT', 'St_FL',
       'St_GA', 'St_IA', 'St_IL', 'St_IN', 'St_KS', 'St_KY', 'St_LA', 'St_MI',
       'St_MN', 'St_MO', 'St_MS', 'St_NE', 'St_NM', 'St_OH', 'St_OK', 'St_PA',
       'St_SD', 'St_TN', 'St_TX', 'St_WI', 'St_WV', 'St_WY',
       'class_Association/Foundation', 'class_Community Club',
       'class_Education Institution', 'class_Food Bank',
       'class_Healtcare and Public Safety', 'class_Library', 'class_Other',
       'class_Religious Organization']]
SVMDataY = pre_learning_data[['participation_in_2023']]

In [50]:
SVX_train, SVX_test, SVY_train, SVY_test = train_test_split(SVMDataX, SVMDataY, test_size=0.3, random_state=1)

In [51]:
# from sklearn.ensemble import SupportVectorClassifier
svclf = SVC(kernel='rbf')
svclf.fit(SVX_train, np.ravel(SVY_train))

SVC()

In [52]:
SVY_pred = svclf.predict(SVX_test)
print('Accuracy of Random Forest classifier on test set: {:.2f}'.format(svclf.score(SVX_test, SVY_test)))

Accuracy of Random Forest classifier on test set: 0.91


In [53]:
from sklearn.metrics import confusion_matrix as sk_confusion_matrix # Import with an alias

confusion_matrix = confusion_matrix(SVY_test, SVY_pred)
print(confusion_matrix)

[[736  27]
 [ 90 403]]


## Gradient Boosting Machines (GBM)

In [54]:
# prompt: based on my pervious methods, could you write GBM for me

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix as sk_confusion_matrix # Import with an alias

# Prepare your data (X and y) as you did for other models
GBMDataX = pre_learning_data[['participation_count', 'participation_in_2017',
       'participation_in_2018', 'participation_in_2019',
       'participation_in_2020', 'participation_in_2021',
       'participation_in_2022', 'sum_total',
       'duration', 'average_period', 'St_AL', 'St_AR', 'St_CT', 'St_FL',
       'St_GA', 'St_IA', 'St_IL', 'St_IN', 'St_KS', 'St_KY', 'St_LA', 'St_MI',
       'St_MN', 'St_MO', 'St_MS', 'St_NE', 'St_NM', 'St_OH', 'St_OK', 'St_PA',
       'St_SD', 'St_TN', 'St_TX', 'St_WI', 'St_WV', 'St_WY',
       'class_Association/Foundation', 'class_Community Club',
       'class_Education Institution', 'class_Food Bank',
       'class_Healtcare and Public Safety', 'class_Library', 'class_Other',
       'class_Religious Organization']]
GBMDataY = pre_learning_data[['participation_in_2023']]

GBMX_train, GBMX_test, GBMY_train, GBMY_test = train_test_split(GBMDataX, GBMDataY, test_size=0.3, random_state=1)

# Initialize and train the Gradient Boosting Classifier
gbm_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=50, random_state=1) # Adjust hyperparameters as needed
gbm_clf.fit(GBMX_train, np.ravel(GBMY_train))

# Make predictions
GBMY_pred = gbm_clf.predict(GBMX_test)

# Evaluate the model
print('Accuracy of Gradient Boosting classifier on test set: {:.2f}'.format(gbm_clf.score(GBMX_test, GBMY_test)))
# Use the alias to call the confusion_matrix function
confusion_matrix_result = sk_confusion_matrix(GBMY_test, GBMY_pred)
print(confusion_matrix_result) # Print the result


Accuracy of Gradient Boosting classifier on test set: 0.93
[[718  45]
 [ 46 447]]


In [65]:
# prompt: then How to use Time Series Cross-Validation to verfiy

import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

tscv = TimeSeriesSplit(n_splits=5)

# Gradient Boosting Classifier
gbm_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=50, random_state=1)

# Cross-validation process
fold = 1
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    gbm_clf.fit(X_train, np.ravel(y_train))

    # Make predictions
    y_pred = gbm_clf.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Fold {fold}: Accuracy = {accuracy:.4f}")
    fold += 1

Fold 1: Accuracy = 1.0000
Fold 2: Accuracy = 0.9986
Fold 3: Accuracy = 1.0000
Fold 4: Accuracy = 1.0000
Fold 5: Accuracy = 1.0000


In [56]:
# prompt: how to use GBMY_pred to estimate new value in dos.csv
new_data = pd.read_csv('dos.csv')
# Assuming you have a new data point in a DataFrame called 'new_data'
# with the same columns as GBMDataX


# Ensure new_data has the same columns as GBMX_train
missing_cols = set(GBMX_train.columns) - set(new_data.columns)
for c in missing_cols:
    new_data[c] = 0
new_data = new_data[GBMX_train.columns]

# Use the trained GBM model to predict the 'participation_in_2023' value
new_prediction = gbm_clf.predict(new_data)
print("Predicted participation in 2023:", new_prediction[0])

# To get probabilities for each class (0 and 1)
new_prediction_proba = gbm_clf.predict_proba(new_data)
print("Predicted probabilities:", new_prediction_proba)

FileNotFoundError: [Errno 2] No such file or directory: 'dos.csv'

In [57]:
# prompt: now I want to build the machine learning to predict the sum amount in next year, could you advise me some machine learning model suite in  my dataset?

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression


# Assuming 'pre_learning_data' is your DataFrame and 'sum_total' is the target variable
# Prepare your features (X) and target variable (y)
X = pre_learning_data.drop('sum_total', axis=1)  # Features (exclude 'sum_total')
y = pre_learning_data['sum_total']  # Target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# 1. Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)
print("Linear Regression:")
print("Mean Squared Error:", mean_squared_error(y_test, lr_predictions))
print("R-squared:", r2_score(y_test, lr_predictions))

# 2. Random Forest Regression
rf_model = RandomForestRegressor(n_estimators=100, random_state=1) # You can tune n_estimators
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
print("\nRandom Forest Regression:")
print("Mean Squared Error:", mean_squared_error(y_test, rf_predictions))
print("R-squared:", r2_score(y_test, rf_predictions))


# 3. Gradient Boosting Regression
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=1) # You can tune n_estimators and other hyperparameters
gb_model.fit(X_train, y_train)
gb_predictions = gb_model.predict(X_test)
print("\nGradient Boosting Regression:")
print("Mean Squared Error:", mean_squared_error(y_test, gb_predictions))
print("R-squared:", r2_score(y_test, gb_predictions))

# Model Selection:
# Compare the Mean Squared Error (MSE) and R-squared values for each model.
# Lower MSE and higher R-squared indicate better performance.
# You can also use other metrics like Mean Absolute Error (MAE).
# Further consider cross-validation to ensure robust performance estimates

Linear Regression:
Mean Squared Error: 163425.00218334724
R-squared: 0.24962598120499524

Random Forest Regression:
Mean Squared Error: 172120.13562898088
R-squared: 0.20970184389191127

Gradient Boosting Regression:
Mean Squared Error: 164695.4308897627
R-squared: 0.24379274466658596
